<a href="https://colab.research.google.com/github/isabelvrielink/Paralinguistic-feature-extraction-for-audio-deepfake-detection/blob/main/Coding_part_1_(testing_models_and_preprocessing_tests).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade kagglehub
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!pip install tensorflow-hub
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!apt-get -qq install -y graphviz && pip install pydot
!pip install cartopy
!pip install pydub

  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.2.4
    Uninstalling kagglehub-0.2.4:
      Successfully uninstalled kagglehub-0.2.4
E: Package 'libfluidsynth1' has no installation candidate
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121920 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for libarchive
  Running setup.py cl

In [ ]:
import kagglehub
import pydot
import cartopy
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import numpy as np
import os
from pydub import AudioSegment
import matplotlib.pyplot as plt

In [ ]:
# Code for downloading the TRILLsson model from Kaggle

In [ ]:
path = kagglehub.model_download("google/trillsson/tensorFlow2/1")
print("Path to model files:", path)

100%|██████████| 19.6M/19.6M [00:02<00:00, 8.60MB/s]
Extracting model files...


Path to model files: /root/.cache/kagglehub/models/google/trillsson/tensorFlow2/1/1


In [ ]:
# Code for loading the TRILLsson model via TensorFlow Hub

In [ ]:
model_url = 'https://tfhub.dev/google/trillsson1/1'
model = hub.KerasLayer(model_url)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


In [ ]:
# split the input audio clips into smaller clips, of 1 second
import os
from pydub import AudioSegment

# file path
file_path = "/path/to/dataset/"

# Directory to save the processed files
output_path = "/path/to/output/"
os.makedirs(output_path, exist_ok=True)

for filename in os.listdir(file_path):
    if filename.endswith(".flac"):
        full_path = os.path.join(file_path, filename)
        audio = AudioSegment.from_file(full_path, format="flac")
        # set it to 1 second
        start_time = (len(audio) // 2) - 500
        end_time = (len(audio) // 2) + 500
        start_time = max(0, start_time)
        end_time = min(len(audio), end_time)
        middle_clip = audio[start_time:end_time]
        # Save the processed clip
        output_file = os.path.join(output_path, filename)
        middle_clip.export(output_file, format="flac")


In [ ]:
!pip install pydub
!pip install librosa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# all following steps are from the testround
# The dev set of ASVspoof2019 is used here to test the paralinguistic feature extraction by TRILLsson

In [ ]:
# TRILLsson test 3
# save all embeddings to 1 map in format "D" in my directory THESIS ISABEL

import tensorflow as tf
import tensorflow_hub as hub
import glob
import numpy as np
import librosa
import os
from datetime import datetime
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# Mount Google Drive if using Google Colab
from google.colab import drive
drive.mount('/content/drive')

model = hub.KerasLayer('https://tfhub.dev/google/trillsson5/1')

seconds = 1

# To store embeddings
embeddings_dir = '/path/to/embeddings/'
os.makedirs(embeddings_dir, exist_ok=True)

# Make sure to point to the correct files directory
files = glob.glob('/path/to/embeddings')
print(f"Found {len(files)} files to process.")

for filename in files:
  if os.path.isfile(filename):
    head, tail = os.path.split(filename)
    filestem = tail[:-5]
    outfilename = f"{embeddings_dir}/{filestem}.txt"  # Ensure 'D' is preserved

    # Check if 'D' is in the filestem to ensure it's correct
    if 'D' not in filestem:
      print(f"Warning: Expected 'D' in filename but found: {filestem}")

    if not os.path.isfile(outfilename):
      y, sr = librosa.load(filename, sr=16000)
      segment = y[0:seconds * sr]
      if len(segment) / sr >= 1:
        y_mono = librosa.to_mono(segment)
        speech = librosa.util.normalize(y_mono)
        audio_sample = tf.convert_to_tensor(tf.reshape(speech, (1, speech.size)))
        embedding = model(audio_sample)['embedding']
        embedding.shape.assert_is_compatible_with([None, 1024])
        np.savetxt(outfilename, embedding.numpy())

# Check saved files in the specific directory
saved_files = glob.glob(f"{embeddings_dir}/*.txt")
print(f"Saved {len(saved_files)} embedding files.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 25137 files to process.
Saved 25012 embedding files.


In [ ]:
import os

# Define the path to the directory
directory = '/path/to/embeddings/'

# Get the list of all files in the directory
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.endswith('.txt')]

# Print the number of files found
print(f"Found {len(files)} files to process.")



Found 25012 files to process.


In [ ]:
# Connect the labels with the embeddings

import os
import numpy as np
import pandas as pd

# Path to both files
embeddings_dir = '/path/to/embeddings'
label_file_path = '/path/to/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt'

# Load labels file and put in dict
labels_dict = {}
with open(label_file_path, 'r') as file:
  for line in file:
    parts = line.strip().split()
    file_id = parts[1] + ".txt"
    label = parts[-1]
    labels_dict[file_id] = 1 if label == 'bonafide' else 0

print(f"Loaded {len(labels_dict)} labels")

# Ensure the embeddings directory exists
if not os.path.exists(embeddings_dir):
    print(f"Embeddings directory {embeddings_dir} does not exist.")
else:
    print(f"Embeddings directory exists: {embeddings_dir}")

# List files in embeddings directory
embeddings_files = os.listdir(embeddings_dir)
print(f"Total embeddings files: {len(embeddings_files)}")

# Sample embeddings files, to check if it works correctly
print("Sample embeddings files:", embeddings_files[:10])

# Sample label identifiers, to check if it works correctly
sample_labels = list(labels_dict.keys())[:10]
print("Sample label identifiers:", sample_labels)

# Tracking pairing
successful_loads = 0
failed_loads = 0

# Filter out duplicates and missing files
matched_files = []
embeddings_dict = {}

for filename in embeddings_files:
  filepath = os.path.join(embeddings_dir, filename)
  if filename in labels_dict:
    try:
      embedding = np.loadtxt(filepath)
      embeddings_dict[filename] = embedding
      matched_files.append(filename)
      successful_loads += 1
    except Exception as e:
      print(f"Failed to load {filename}: {e}")
      failed_loads += 1

print(f"Number of matched files: {len(matched_files)}")
print(f"Loaded {successful_loads} embeddings")
print(f"Failed to load {failed_loads} embeddings")

# Filtered out unmatched labels
filtered_labels_dict = {k: v for k, v in labels_dict.items() if k in matched_files}

print(f"Filtered to {len(filtered_labels_dict)} matched labels")

# Create dataframe
data = {
    'embedding': [embeddings_dict[filename] for filename in matched_files],
    'label': [filtered_labels_dict[filename] for filename in matched_files]}

df = pd.DataFrame(data)
print(df.head())


Loaded 24844 labels
Embeddings directory exists: /content/drive/My Drive/THESIS ISABEL/embeddings
Total embeddings files: 25012
Sample embeddings files: ['LA_D_3447687.txt', 'LA_D_2896345.txt', 'LA_D_7114286.txt', 'LA_D_3510615.txt', 'LA_D_4094129.txt', 'LA_D_3709416.txt', 'LA_D_9089312.txt', 'LA_D_6751883.txt', 'LA_D_2596805.txt', 'LA_D_9685224.txt']
Sample label identifiers: ['LA_D_1047731.txt', 'LA_D_1105538.txt', 'LA_D_1125976.txt', 'LA_D_1293230.txt', 'LA_D_1340209.txt', 'LA_D_1376638.txt', 'LA_D_1377420.txt', 'LA_D_1403371.txt', 'LA_D_1590530.txt', 'LA_D_1595036.txt']
Number of matched files: 24720
Loaded 24720 embeddings
Failed to load 0 embeddings
Filtered to 24720 matched labels
                                           embedding  label
0  [-1.4889453649520874, -0.8926263451576233, -0....      0
1  [-0.7889299988746643, -0.18677853047847748, -0...      1
2  [-0.949458658695221, -0.6341816186904907, -1.1...      0
3  [-1.273923635482788, -1.2051512002944946, -0.6...      0
4  